In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/deepstab/stable/38.avi
/kaggle/input/deepstab/stable/17.avi
/kaggle/input/deepstab/stable/18.avi
/kaggle/input/deepstab/stable/4.avi
/kaggle/input/deepstab/stable/51.avi
/kaggle/input/deepstab/stable/57.avi
/kaggle/input/deepstab/stable/32.avi
/kaggle/input/deepstab/stable/28.avi
/kaggle/input/deepstab/stable/1.avi
/kaggle/input/deepstab/stable/58.avi
/kaggle/input/deepstab/stable/53.avi
/kaggle/input/deepstab/stable/7.avi
/kaggle/input/deepstab/stable/24.avi
/kaggle/input/deepstab/stable/11.avi
/kaggle/input/deepstab/stable/34.avi
/kaggle/input/deepstab/stable/13.avi
/kaggle/input/deepstab/stable/46.avi
/kaggle/input/deepstab/stable/47.avi
/kaggle/input/deepstab/stable/16.avi
/kaggle/input/deepstab/stable/3.avi
/kaggle/input/deepstab/stable/35.avi
/kaggle/input/deepstab/stable/41.avi
/kaggle/input/deepstab/stable/36.avi
/kaggle/input/deepstab/stable/30.avi
/kaggle/input/deepstab/stable/26.avi
/kaggle/input/deepstab/stable/50.avi
/kaggle/input/deepstab/stable/42.avi
/kagg

In [1]:
import os
import glob
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models
from tqdm import tqdm
import math
from skimage.metrics import structural_similarity as ssim_metric

# ==========================================
# 1. HELPER FUNCTIONS
# ==========================================
def calculate_ssim_cropped(img1, img2, crop_percent=0.1):
    """
    Crops the outer 10% of the image before calculating SSIM.
    This ignores the black borders caused by stabilization warping.
    img1, img2: Numpy arrays (H, W, C), 0-255, uint8
    """
    h, w = img1.shape[0], img1.shape[1]
    
    # Calculate crop margins
    h_crop = int(h * crop_percent)
    w_crop = int(w * crop_percent)
    
    # Crop center (Handle edge case if crop is 0)
    if h_crop > 0 and w_crop > 0:
        img1_center = img1[h_crop:h-h_crop, w_crop:w-w_crop]
        img2_center = img2[h_crop:h-h_crop, w_crop:w-w_crop]
    else:
        img1_center = img1
        img2_center = img2
    
    # Convert to gray for SSIM (Safe check for channel dim)
    if len(img1_center.shape) == 3 and img1_center.shape[2] == 3:
        gray1 = cv2.cvtColor(img1_center, cv2.COLOR_RGB2GRAY)
        gray2 = cv2.cvtColor(img2_center, cv2.COLOR_RGB2GRAY)
    else:
        gray1, gray2 = img1_center, img2_center

    return ssim_metric(gray1, gray2, data_range=255)

# ==========================================
# 2. CONFIGURATION
# ==========================================
CONFIG = {
    'EPOCHS': 200,
    'BATCH_SIZE': 2,        
    'CLIP_LEN': 32,         
    'RESIZE': (192, 192),   
    'LR_G': 1e-4,           
    'LR_D': 1e-5,           
    'DEVICE': torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    'KAGGLE_ROOT': "/kaggle/input/deepstab" 
}

print(f"🔥 Running on {CONFIG['DEVICE']} | Epochs: {CONFIG['EPOCHS']}")

# ==========================================
# 3. DATASET
# ==========================================
class DeepStabDataset(Dataset):
    def __init__(self, root_dir, type='train'):
        self.resize = CONFIG['RESIZE']
        self.clip_len = CONFIG['CLIP_LEN']
        
        possible_roots = [os.path.join(root_dir, type), root_dir]
        self.unstable_paths = []
        self.stable_paths = []
        
        for r in possible_roots:
            u_path = os.path.join(r, 'unstable')
            s_path = os.path.join(r, 'stable')
            if os.path.exists(u_path) and os.path.exists(s_path):
                valid_exts = ('.mp4', '.avi', '.mov')
                self.unstable_paths = sorted([f for f in glob.glob(os.path.join(u_path, '*')) if f.endswith(valid_exts)])
                self.stable_paths = sorted([f for f in glob.glob(os.path.join(s_path, '*')) if f.endswith(valid_exts)])
                break
        
        if len(self.unstable_paths) == 0:
            print(f"⚠️ WARNING: No videos found in {root_dir}. Creating DUMMY data.")
            self.unstable_paths = ["dummy_u.mp4"] * 10
            self.stable_paths = ["dummy_s.mp4"] * 10
        else:
            print(f"✅ Loaded {len(self.unstable_paths)} pairs from {root_dir}")

    def compute_optical_flow(self, frames):
        flows = []
        prev_gray = cv2.cvtColor(frames[0], cv2.COLOR_RGB2GRAY)
        flows.append(np.zeros((self.resize[1], self.resize[0], 2), dtype=np.float32))
        
        for i in range(1, len(frames)):
            curr_gray = cv2.cvtColor(frames[i], cv2.COLOR_RGB2GRAY)
            flow = cv2.calcOpticalFlowFarneback(prev_gray, curr_gray, None, 
                                                0.5, 3, 15, 3, 5, 1.2, 0)
            flows.append(flow)
            prev_gray = curr_gray
        return np.array(flows)

    def load_video_clip(self, path):
        if path == "dummy_u.mp4" or path == "dummy_s.mp4":
            return np.random.randint(0, 255, (self.clip_len, self.resize[1], self.resize[0], 3), dtype=np.uint8)

        cap = cv2.VideoCapture(path)
        frames = []
        while True:
            ret, frame = cap.read()
            if not ret: break
            frame = cv2.resize(frame, self.resize)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame)
            if len(frames) >= self.clip_len: break
        cap.release()
        
        if len(frames) < self.clip_len:
            padding = [frames[-1]] * (self.clip_len - len(frames))
            frames.extend(padding)
            
        return np.array(frames)

    def __getitem__(self, idx):
        unstable = self.load_video_clip(self.unstable_paths[idx])
        stable = self.load_video_clip(self.stable_paths[idx])
        flow = self.compute_optical_flow(unstable)
        
        u_rgb = torch.from_numpy(unstable).permute(3, 0, 1, 2).float() / 255.0
        s_rgb = torch.from_numpy(stable).permute(3, 0, 1, 2).float() / 255.0
        u_flow = torch.from_numpy(flow).permute(3, 0, 1, 2).float()
        
        input_tensor = torch.cat([u_rgb, u_flow], dim=0)
        return input_tensor, s_rgb, u_rgb

    def __len__(self):
        return len(self.unstable_paths)

# ==========================================
# 4. MODELS
# ==========================================
class StabGenerator(nn.Module):
    def __init__(self):
        super(StabGenerator, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv3d(5, 64, 3, padding=1), nn.ReLU(), nn.MaxPool3d((1,2,2)),
            nn.Conv3d(64, 128, 3, padding=1), nn.ReLU(), nn.MaxPool3d((1,2,2)),
            nn.Conv3d(128, 256, 3, padding=1), nn.ReLU(), nn.MaxPool3d((1,2,2)),
            nn.AdaptiveAvgPool3d((CONFIG['CLIP_LEN'], 1, 1))
        )
        self.regressor = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * CONFIG['CLIP_LEN'], 512), nn.ReLU(),
            nn.Linear(512, CONFIG['CLIP_LEN'] * 6)
        )

    def forward(self, x_5ch, x_3ch_raw):
        b, _, t, h, w = x_5ch.shape
        features = self.encoder(x_5ch)
        theta = self.regressor(features).view(-1, 2, 3)
        
        identity = torch.tensor([1, 0, 0, 0, 1, 0], dtype=torch.float32, device=x_5ch.device)
        identity = identity.view(1, 2, 3).repeat(theta.shape[0], 1, 1)
        final_theta = identity + (theta * 0.1)
        
        x_flat = x_3ch_raw.permute(0, 2, 1, 3, 4).reshape(-1, 3, h, w)
        grid = F.affine_grid(final_theta, x_flat.size(), align_corners=True)
        stabilized = F.grid_sample(x_flat, grid, align_corners=True)
        
        return stabilized.view(b, t, 3, h, w).permute(0, 2, 1, 3, 4), final_theta

class StabDiscriminator(nn.Module):
    def __init__(self):
        super(StabDiscriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv3d(3, 64, 4, 2, 1), nn.LeakyReLU(0.2),
            nn.Conv3d(64, 128, 4, 2, 1), nn.BatchNorm3d(128), nn.LeakyReLU(0.2),
            nn.Conv3d(128, 256, 4, 2, 1), nn.BatchNorm3d(256), nn.LeakyReLU(0.2),
            nn.Conv3d(256, 1, 4, 1, 0), nn.Sigmoid()
        )
    def forward(self, x):
        return self.model(x).view(-1, 1)

class VGGPerceptualLoss(nn.Module):
    def __init__(self):
        super(VGGPerceptualLoss, self).__init__()
        vgg = models.vgg19(weights=models.VGG19_Weights.DEFAULT).features
        self.slice = nn.Sequential(*list(vgg.children())[:12]).eval()
        for param in self.slice.parameters(): param.requires_grad = False

    def forward(self, est, target):
        b, c, t, h, w = est.shape
        est_flat = est.permute(0, 2, 1, 3, 4).reshape(-1, c, h, w)
        target_flat = target.permute(0, 2, 1, 3, 4).reshape(-1, c, h, w)
        return F.mse_loss(self.slice(est_flat), self.slice(target_flat))

# ==========================================
# 5. TRAINING LOOP
# ==========================================
def train_full_pipeline():
    dataset = DeepStabDataset(root_dir=CONFIG['KAGGLE_ROOT'])
    dataloader = DataLoader(dataset, batch_size=CONFIG['BATCH_SIZE'], shuffle=True, num_workers=2)
    
    netG = StabGenerator().to(CONFIG['DEVICE'])
    netD = StabDiscriminator().to(CONFIG['DEVICE'])
    vgg_loss = VGGPerceptualLoss().to(CONFIG['DEVICE'])
    
    optG = optim.Adam(netG.parameters(), lr=CONFIG['LR_G'], betas=(0.5, 0.999))
    optD = optim.Adam(netD.parameters(), lr=CONFIG['LR_D'], betas=(0.5, 0.999))
    
    L1_loss = nn.L1Loss()
    BCE_loss = nn.BCELoss()
    
    best_psnr = 0.0

    print("🚀 Training Started...")
    
    for epoch in range(CONFIG['EPOCHS']):
        netG.train()
        loop = tqdm(dataloader, desc=f"Epoch {epoch+1}/{CONFIG['EPOCHS']}")
        
        for i, (in_5ch, target_3ch, in_3ch_raw) in enumerate(loop):
            in_5ch = in_5ch.to(CONFIG['DEVICE'])
            target_3ch = target_3ch.to(CONFIG['DEVICE'])
            in_3ch_raw = in_3ch_raw.to(CONFIG['DEVICE'])
            
            # --- Train Discriminator ---
            netD.zero_grad()
            real_score = netD(target_3ch)
            real_labels = torch.ones_like(real_score)
            loss_d_real = BCE_loss(real_score, real_labels)
            
            fake_video, transforms = netG(in_5ch, in_3ch_raw)
            fake_score = netD(fake_video.detach())
            fake_labels = torch.zeros_like(fake_score)
            loss_d_fake = BCE_loss(fake_score, fake_labels)
            
            loss_D = (loss_d_real + loss_d_fake) / 2
            loss_D.backward()
            optD.step()
            
            # --- Train Generator ---
            netG.zero_grad()
            out_score = netD(fake_video)
            loss_gan = BCE_loss(out_score, real_labels) # Fool D
            
            loss_pixel = L1_loss(fake_video, target_3ch)
            loss_vgg = vgg_loss(fake_video, target_3ch)
            
            # Smoothness Loss
            b, t_all, _ = transforms.view(-1, CONFIG['CLIP_LEN'], 6).shape
            transforms_rs = transforms.view(b, CONFIG['CLIP_LEN'], 6)
            diff1 = transforms_rs[:, 1:] - transforms_rs[:, :-1]
            diff2 = diff1[:, 1:] - diff1[:, :-1]
            loss_smooth = torch.mean(diff2 ** 2)

            loss_G = (1.0 * loss_gan) + (100.0 * loss_pixel) + (10.0 * loss_vgg) + (50.0 * loss_smooth)
            
            loss_G.backward()
            optG.step()
            
            loop.set_postfix(Loss_G=loss_G.item(), Loss_D=loss_D.item())
        
        # --- End of Epoch: Eval with Cropped SSIM ---
        with torch.no_grad():
            # 1. PSNR Calculation
            mse = F.mse_loss(fake_video, target_3ch).item()
            psnr = 20 * math.log10(1.0 / math.sqrt(mse)) if mse > 0 else 0
            
            # 2. SSIM Calculation (Cropped)
            # Take first item in batch, middle frame
            # Dimensions: (Batch, 3, T, H, W) -> (3, H, W) -> (H, W, 3)
            mid_idx = CONFIG['CLIP_LEN'] // 2
            
            fake_np = fake_video[0, :, mid_idx, :, :].permute(1, 2, 0).cpu().numpy() * 255.0
            targ_np = target_3ch[0, :, mid_idx, :, :].permute(1, 2, 0).cpu().numpy() * 255.0
            
            fake_np = np.clip(fake_np, 0, 255).astype(np.uint8)
            targ_np = np.clip(targ_np, 0, 255).astype(np.uint8)
            
            # Call our helper function
            ssim_score = calculate_ssim_cropped(fake_np, targ_np, crop_percent=0.1)
            
            if psnr > best_psnr:
                best_psnr = psnr
                torch.save(netG.state_dict(), "best_stab_gan.pth")
                print(f"🌟 Epoch {epoch+1}: New Best PSNR: {best_psnr:.2f} dB | SSIM (Cropped): {ssim_score:.4f}")
            else:
                print(f"   Epoch {epoch+1}: PSNR: {psnr:.2f} dB | SSIM (Cropped): {ssim_score:.4f}")
                
    torch.save(netG.state_dict(), "final_stab_gan.pth")
    print("✅ Training Complete.")

if __name__ == "__main__":
    train_full_pipeline()

🔥 Running on cuda | Epochs: 200
✅ Loaded 61 pairs from /kaggle/input/deepstab


Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:02<00:00, 249MB/s] 


🚀 Training Started...


Epoch 1/200: 100%|██████████| 31/31 [00:38<00:00,  1.23s/it, Loss_D=0.668, Loss_G=35.3]


🌟 Epoch 1: New Best PSNR: 12.68 dB | SSIM (Cropped): 0.3143


...

   Epoch 198: PSNR: 16.46 dB | SSIM (Cropped): 0.4068


Epoch 199/200: 100%|██████████| 31/31 [00:44<00:00,  1.44s/it, Loss_D=0.0115, Loss_G=39.1] 


   Epoch 199: PSNR: 16.04 dB | SSIM (Cropped): 0.2575


Epoch 200/200: 100%|██████████| 31/31 [00:44<00:00,  1.44s/it, Loss_D=0.00557, Loss_G=39.5] 

   Epoch 200: PSNR: 14.68 dB | SSIM (Cropped): 0.2582
✅ Training Complete.


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import cv2
import numpy as np
import os
import glob
import math
from tqdm import tqdm
from skimage.metrics import structural_similarity as ssim_metric

# ==========================================
# 1. SETUP & CONFIG
# ==========================================
CONFIG = {
    'RESIZE': (192, 192),
    'CLIP_LEN': 32,
    'DEVICE': torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    'MODEL_PATH': "best_stab_gan.pth",  # Path to your saved model
    'DATA_ROOT': "/kaggle/input/deepstab" # Path to your data
}

# ==========================================
# 2. MODEL DEFINITION (Must match training)
# ==========================================
class StabGenerator(nn.Module):
    def __init__(self):
        super(StabGenerator, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv3d(5, 64, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool3d((1,2,2)),
            nn.Conv3d(64, 128, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool3d((1,2,2)),
            nn.Conv3d(128, 256, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool3d((1,2,2)),
            nn.AdaptiveAvgPool3d((CONFIG['CLIP_LEN'], 1, 1))
        )
        self.regressor = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * CONFIG['CLIP_LEN'], 512),
            nn.ReLU(),
            nn.Linear(512, CONFIG['CLIP_LEN'] * 6)
        )

    def forward(self, x_5ch, x_3ch_raw):
        b, _, t, h, w = x_5ch.shape
        features = self.encoder(x_5ch)
        theta = self.regressor(features).view(-1, 2, 3)
        
        identity = torch.tensor([1, 0, 0, 0, 1, 0], dtype=torch.float32, device=x_5ch.device)
        identity = identity.view(1, 2, 3).repeat(theta.shape[0], 1, 1)
        final_theta = identity + (theta * 0.1)
        
        x_flat = x_3ch_raw.permute(0, 2, 1, 3, 4).reshape(-1, 3, h, w)
        grid = F.affine_grid(final_theta, x_flat.size(), align_corners=True)
        stabilized = F.grid_sample(x_flat, grid, align_corners=True)
        
        return stabilized.view(b, t, 3, h, w).permute(0, 2, 1, 3, 4)

# ==========================================
# 3. METRIC FUNCTIONS
# ==========================================
def calculate_psnr(img1, img2):
    """Peak Signal-to-Noise Ratio (Higher is Better)"""
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0: return 100
    return 20 * math.log10(255.0 / math.sqrt(mse))

def calculate_ssim(img1, img2):
    """Structural Similarity (Higher is Better, Max 1.0)"""
    # SSIM works best on grayscale
    gray1 = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)
    return ssim_metric(gray1, gray2, data_range=255)

def calculate_stability(frames):
    """
    Calculates the 'shakiness' score of a video clip.
    Lower score = More Stable.
    """
    motion_score = 0
    # Calculate simple frame-to-frame pixel difference
    for i in range(1, len(frames)):
        diff = np.mean(np.abs(frames[i].astype(np.float32) - frames[i-1].astype(np.float32)))
        motion_score += diff
    return motion_score / len(frames)

# ==========================================
# 4. DATA LOADER (Simplified for Eval)
# ==========================================
class EvalDataset(Dataset):
    def __init__(self, root_dir):
        self.resize = CONFIG['RESIZE']
        self.clip_len = CONFIG['CLIP_LEN']
        
        # Try finding data
        unstable_dir = os.path.join(root_dir, 'train', 'unstable')
        stable_dir = os.path.join(root_dir, 'train', 'stable')
        if not os.path.exists(unstable_dir): # Try alternate path
            unstable_dir = os.path.join(root_dir, 'unstable')
            stable_dir = os.path.join(root_dir, 'stable')
            
        self.unstable_paths = sorted(glob.glob(os.path.join(unstable_dir, '*')))[:20] # Test on 20 clips
        self.stable_paths = sorted(glob.glob(os.path.join(stable_dir, '*')))[:20]

    def _load_clip(self, path):
        cap = cv2.VideoCapture(path)
        frames = []
        while True:
            ret, frame = cap.read()
            if not ret: break
            frame = cv2.resize(frame, self.resize)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame)
            if len(frames) >= self.clip_len: break
        cap.release()
        # Padding
        if len(frames) < self.clip_len:
            frames.extend([frames[-1]] * (self.clip_len - len(frames)))
        return np.array(frames)

    def _compute_flow(self, frames):
        prev = cv2.cvtColor(frames[0], cv2.COLOR_RGB2GRAY)
        flows = [np.zeros((self.resize[1], self.resize[0], 2), dtype=np.float32)]
        for i in range(1, len(frames)):
            curr = cv2.cvtColor(frames[i], cv2.COLOR_RGB2GRAY)
            flow = cv2.calcOpticalFlowFarneback(prev, curr, None, 0.5, 3, 15, 3, 5, 1.2, 0)
            flows.append(flow)
            prev = curr
        return np.array(flows)

    def __getitem__(self, idx):
        unstable = self._load_clip(self.unstable_paths[idx])
        stable = self._load_clip(self.stable_paths[idx])
        flow = self._compute_flow(unstable)
        
        u_rgb = torch.from_numpy(unstable).permute(3, 0, 1, 2).float() / 255.0
        s_rgb = torch.from_numpy(stable).permute(3, 0, 1, 2).float() / 255.0
        u_flow = torch.from_numpy(flow).permute(3, 0, 1, 2).float()
        
        return torch.cat([u_rgb, u_flow], dim=0), s_rgb, u_rgb

    def __len__(self):
        return len(self.unstable_paths)

# ==========================================
# 5. MAIN EVALUATION LOOP
# ==========================================
def run_evaluation():
    print(f"📊 Loading Model from: {CONFIG['MODEL_PATH']}")
    
    # Load Model
    model = StabGenerator().to(CONFIG['DEVICE'])
    try:
        model.load_state_dict(torch.load(CONFIG['MODEL_PATH'], map_location=CONFIG['DEVICE']))
    except:
        print("⚠️ Warning: Model weights not found. Running with random weights (Results will be bad).")
    
    model.eval()
    
    # Load Data
    dataset = EvalDataset(CONFIG['DATA_ROOT'])
    loader = DataLoader(dataset, batch_size=1, shuffle=False)
    
    avg_psnr, avg_ssim = [], []
    stability_gain = []
    
    print(f"🚀 Testing on {len(dataset)} clips...")
    
    with torch.no_grad():
        for i, (in_5ch, target, in_raw) in enumerate(tqdm(loader)):
            in_5ch = in_5ch.to(CONFIG['DEVICE'])
            target = target.to(CONFIG['DEVICE'])
            in_raw = in_raw.to(CONFIG['DEVICE'])
            
            # 1. Inference
            stabilized_tensor = model(in_5ch, in_raw)
            
            # 2. Convert to Numpy for Metrics (uint8 0-255)
            # Remove Batch dim -> (3, T, H, W) -> (T, H, W, 3)
            res = stabilized_tensor[0].permute(1, 2, 3, 0).cpu().numpy() * 255.0
            gt = target[0].permute(1, 2, 3, 0).cpu().numpy() * 255.0
            orig = in_raw[0].permute(1, 2, 3, 0).cpu().numpy() * 255.0
            
            res = np.clip(res, 0, 255).astype(np.uint8)
            gt = np.clip(gt, 0, 255).astype(np.uint8)
            orig = np.clip(orig, 0, 255).astype(np.uint8)
            
            # 3. Compute Metrics per frame
            clip_psnr = np.mean([calculate_psnr(res[f], gt[f]) for f in range(32)])
            clip_ssim = np.mean([calculate_ssim(res[f], gt[f]) for f in range(32)])
            
            # 4. Compute Stability
            stab_score_orig = calculate_stability(orig)
            stab_score_res = calculate_stability(res)
            # % Improvement
            gain = ((stab_score_orig - stab_score_res) / (stab_score_orig + 1e-6)) * 100
            
            avg_psnr.append(clip_psnr)
            avg_ssim.append(clip_ssim)
            stability_gain.append(gain)

    # ==========================================
    # 6. FINAL REPORT
    # ==========================================
    print("\n" + "="*40)
    print("📈 FINAL EVALUATION MATRIX")
    print("="*40)
    print(f"1. PSNR (Quality)   : {np.mean(avg_psnr):.2f} dB  (>25 is Good)")
    print(f"2. SSIM (Similarity): {np.mean(avg_ssim):.4f}     (>0.8 is Good)")
    print(f"3. Stability Gain   : {np.mean(stability_gain):.2f}%   (>20% is Good)")
    print("="*40)

    if np.mean(avg_psnr) > 28 and np.mean(stability_gain) > 30:
        print("✅ VERDICT: Excellent Model! Ready for production.")
    elif np.mean(avg_psnr) > 20:
        print("⚠️ VERDICT: Decent results. Train longer for better smoothness.")
    else:
        print("❌ VERDICT: Low accuracy. Check dataset or increase Epochs.")

if __name__ == "__main__":
    run_evaluation()

📊 Loading Model from: best_stab_gan.pth
🚀 Testing on 20 clips...


100%|██████████| 20/20 [00:16<00:00,  1.21it/s]


📈 FINAL EVALUATION MATRIX
1. PSNR (Quality)   : 28.57 dB  (>25 is Good)
2. SSIM (Similarity): 0.3074     (>0.8 is Good)
3. Stability Gain   : 33.58%   (>20% is Good)
✅ VERDICT: Excellent Model! Ready for production.


In [5]:
import os
import glob
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import math
from skimage.metrics import structural_similarity as ssim_metric

# ==========================================
# 1. CONFIGURATION
# ==========================================
CONFIG = {
    'CLIP_LEN': 32,
    'RESIZE': (192, 192),
    'DEVICE': torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    'KAGGLE_ROOT': "/kaggle/input/deepstab/stable",  # Path to your dataset
    'MODEL_PATH': "best_stab_gan.pth"         # Path to your saved model
}

# ==========================================
# 2. METRIC FUNCTIONS
# ==========================================
def calculate_psnr(img1, img2):
    """Calculates Peak Signal-to-Noise Ratio (Higher is Better)"""
    mse = np.mean((img1.astype(np.float32) - img2.astype(np.float32)) ** 2)
    if mse == 0: return 100
    return 20 * math.log10(255.0 / math.sqrt(mse))

def calculate_ssim_cropped(img1, img2, crop_percent=0.1):
    """Calculates SSIM on the center of the image to ignore black borders."""
    h, w = img1.shape[0], img1.shape[1]
    h_crop, w_crop = int(h * crop_percent), int(w * crop_percent)
    
    # Handle small images or 0 crop
    if h_crop == 0 or w_crop == 0:
        c1, c2 = img1, img2
    else:
        c1 = img1[h_crop:h-h_crop, w_crop:w-w_crop]
        c2 = img2[h_crop:h-h_crop, w_crop:w-w_crop]
    
    # Convert to grayscale for SSIM
    if len(c1.shape) == 3:
        gray1 = cv2.cvtColor(c1, cv2.COLOR_RGB2GRAY)
        gray2 = cv2.cvtColor(c2, cv2.COLOR_RGB2GRAY)
    else:
        gray1, gray2 = c1, c2
    
    return ssim_metric(gray1, gray2, data_range=255)

def calculate_stability_score(frames):
    """
    Calculates average inter-frame pixel difference.
    Lower score = Less movement/shake.
    """
    diffs = []
    for i in range(1, len(frames)):
        # L1 diff between consecutive frames
        diff = np.mean(np.abs(frames[i].astype(np.float32) - frames[i-1].astype(np.float32)))
        diffs.append(diff)
    return np.mean(diffs) if len(diffs) > 0 else 0.0

# ==========================================
# 3. TEST DATASET
# ==========================================
class TestDataset(Dataset):
    def __init__(self, root_dir):
        self.resize = CONFIG['RESIZE']
        self.clip_len = CONFIG['CLIP_LEN']
        
        # 1. Look for 'test' folder first, otherwise use 'train'
        test_dir = os.path.join(root_dir, 'test')
        if not os.path.exists(test_dir):
            test_dir = os.path.join(root_dir, 'train')
            print(f"⚠️ 'test' folder not found. Evaluating on '{test_dir}' instead.")
        
        # 2. Find paths
        u_path = os.path.join(test_dir, 'unstable')
        s_path = os.path.join(test_dir, 'stable')
        
        # Handle cases where path structure is different (e.g. deepstab/unstable directly)
        if not os.path.exists(u_path):
             u_path = os.path.join(root_dir, 'unstable')
             s_path = os.path.join(root_dir, 'stable')

        valid_exts = ('.mp4', '.avi', '.mov')
        self.unstable_paths = sorted([f for f in glob.glob(os.path.join(u_path, '*')) if f.endswith(valid_exts)])
        self.stable_paths = sorted([f for f in glob.glob(os.path.join(s_path, '*')) if f.endswith(valid_exts)])
        
        # Limit evaluation to 50 videos max to save time (Optional)
        self.unstable_paths = self.unstable_paths[:50]
        self.stable_paths = self.stable_paths[:50]
        
        print(f"📊 Evaluator Loaded: {len(self.unstable_paths)} pairs.")

    def _process_clip(self, path):
        cap = cv2.VideoCapture(path)
        frames = []
        while True:
            ret, frame = cap.read()
            if not ret: break
            frame = cv2.resize(frame, self.resize)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame)
            if len(frames) >= self.clip_len: break
        cap.release()
        
        if len(frames) < self.clip_len:
            frames.extend([frames[-1]] * (self.clip_len - len(frames)))
            
        frames = np.array(frames)
        
        # Compute Flow
        flows = []
        prev = cv2.cvtColor(frames[0], cv2.COLOR_RGB2GRAY)
        flows.append(np.zeros((self.resize[1], self.resize[0], 2), dtype=np.float32))
        for i in range(1, len(frames)):
            curr = cv2.cvtColor(frames[i], cv2.COLOR_RGB2GRAY)
            flow = cv2.calcOpticalFlowFarneback(prev, curr, None, 0.5, 3, 15, 3, 5, 1.2, 0)
            flows.append(flow)
            prev = curr
            
        u_rgb = torch.from_numpy(frames).permute(3, 0, 1, 2).float() / 255.0
        u_flow = torch.from_numpy(np.array(flows)).permute(3, 0, 1, 2).float()
        
        input_5ch = torch.cat([u_rgb, u_flow], dim=0)
        return input_5ch, u_rgb, frames # Return raw frames for metrics

    def __getitem__(self, idx):
        # Load Inputs
        in_5ch, in_raw_tensor, in_np = self._process_clip(self.unstable_paths[idx])
        # Load Ground Truth (Stable)
        _, gt_raw_tensor, gt_np = self._process_clip(self.stable_paths[idx])
        
        return in_5ch, in_raw_tensor, gt_raw_tensor

    def __len__(self):
        return len(self.unstable_paths)

# ==========================================
# 4. MODEL ARCHITECTURE (Must match training)
# ==========================================
class StabGenerator(nn.Module):
    def __init__(self):
        super(StabGenerator, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv3d(5, 64, 3, padding=1), nn.ReLU(), nn.MaxPool3d((1,2,2)),
            nn.Conv3d(64, 128, 3, padding=1), nn.ReLU(), nn.MaxPool3d((1,2,2)),
            nn.Conv3d(128, 256, 3, padding=1), nn.ReLU(), nn.MaxPool3d((1,2,2)),
            nn.AdaptiveAvgPool3d((CONFIG['CLIP_LEN'], 1, 1))
        )
        self.regressor = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * CONFIG['CLIP_LEN'], 512), nn.ReLU(),
            nn.Linear(512, CONFIG['CLIP_LEN'] * 6)
        )

    def forward(self, x_5ch, x_3ch_raw):
        b, _, t, h, w = x_5ch.shape
        features = self.encoder(x_5ch)
        theta = self.regressor(features).view(-1, 2, 3)
        identity = torch.tensor([1, 0, 0, 0, 1, 0], dtype=torch.float32, device=x_5ch.device)
        identity = identity.view(1, 2, 3).repeat(theta.shape[0], 1, 1)
        final_theta = identity + (theta * 0.1)
        x_flat = x_3ch_raw.permute(0, 2, 1, 3, 4).reshape(-1, 3, h, w)
        grid = F.affine_grid(final_theta, x_flat.size(), align_corners=True)
        stabilized = F.grid_sample(x_flat, grid, align_corners=True)
        return stabilized.view(b, t, 3, h, w).permute(0, 2, 1, 3, 4)

# ==========================================
# 5. MAIN EVALUATION LOOP
# ==========================================
def evaluate_model():
    print(f"🚀 Starting Evaluation using model: {CONFIG['MODEL_PATH']}")
    
    # 1. Load Model
    model = StabGenerator().to(CONFIG['DEVICE'])
    if os.path.exists(CONFIG['MODEL_PATH']):
        model.load_state_dict(torch.load(CONFIG['MODEL_PATH'], map_location=CONFIG['DEVICE']))
        print("✅ Weights loaded successfully.")
    else:
        print("❌ Error: Model path not found! Check your filename.")
        return

    model.eval()
    
    # 2. Load Data
    try:
        test_dataset = TestDataset(CONFIG['KAGGLE_ROOT'])
    except Exception as e:
        print(f"❌ Data Error: {e}")
        return

    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    
    # Accumulators
    avg_psnr = []
    avg_ssim = []
    avg_stab_gain = []
    
    # 3. Inference Loop
    with torch.no_grad():
        for i, (in_5ch, in_raw, gt_raw) in enumerate(tqdm(test_loader, desc="Testing")):
            in_5ch = in_5ch.to(CONFIG['DEVICE'])
            in_raw = in_raw.to(CONFIG['DEVICE'])
            
            # Predict
            stabilized_tensor = model(in_5ch, in_raw)
            
            # Convert to Numpy (Batch 0) -> (T, H, W, C)
            stab_np = stabilized_tensor[0].permute(1, 2, 3, 0).cpu().numpy() * 255.0
            gt_np = gt_raw[0].permute(1, 2, 3, 0).cpu().numpy() * 255.0
            in_np = in_raw[0].permute(1, 2, 3, 0).cpu().numpy() * 255.0
            
            stab_np = np.clip(stab_np, 0, 255).astype(np.uint8)
            gt_np = np.clip(gt_np, 0, 255).astype(np.uint8)
            in_np = np.clip(in_np, 0, 255).astype(np.uint8)
            
            # --- CALCULATE METRICS PER FRAME ---
            clip_psnr = []
            clip_ssim = []
            
            for f in range(len(stab_np)):
                p = calculate_psnr(stab_np[f], gt_np[f])
                s = calculate_ssim_cropped(stab_np[f], gt_np[f])
                clip_psnr.append(p)
                clip_ssim.append(s)
            
            # --- CALCULATE STABILITY ---
            score_input = calculate_stability_score(in_np)
            score_output = calculate_stability_score(stab_np)
            
            # Gain % (How much closer to 0 did we get?)
            gain = ((score_input - score_output) / (score_input + 1e-6)) * 100
            
            avg_psnr.append(np.mean(clip_psnr))
            avg_ssim.append(np.mean(clip_ssim))
            avg_stab_gain.append(gain)

    # 4. Final Report
    final_psnr = np.mean(avg_psnr)
    final_ssim = np.mean(avg_ssim)
    final_gain = np.mean(avg_stab_gain)
    
    print("\n" + "="*40)
    print("📋 FINAL TEST REPORT")
    print("="*40)
    print(f"1. PSNR (Image Quality) : {final_psnr:.2f} dB   [Target: >25]")
    print(f"2. SSIM (Structural)    : {final_ssim:.4f}      [Target: >0.7]")
    print(f"3. Stability Improved   : {final_gain:.2f} %     [Target: >20%]")
    print("="*40)
    
    if final_psnr > 25.0 and final_gain > 20.0:
        print("✅ SUCCESS: The model effectively stabilizes video while keeping quality.")
    else:
        print("⚠️ NOTE: Results are low? Check if 'stable' ground truth has cropping/zoom differences.")

if __name__ == "__main__":
    evaluate_model()

🚀 Starting Evaluation using model: best_stab_gan.pth
✅ Weights loaded successfully.
⚠️ 'test' folder not found. Evaluating on '/kaggle/input/deepstab/stable/train' instead.
📊 Evaluator Loaded: 0 pairs.


Testing: 0it [00:00, ?it/s]


📋 FINAL TEST REPORT
1. PSNR (Image Quality) : nan dB   [Target: >25]
2. SSIM (Structural)    : nan      [Target: >0.7]
3. Stability Improved   : nan %     [Target: >20%]
⚠️ NOTE: Results are low? Check if 'stable' ground truth has cropping/zoom differences.
